In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import os
import tensorflow as tf
import numpy as np
from sklearn import model_selection
from tqdm.notebook import tqdm

BASE_DIR = '../../../'
import sys
sys.path.append(BASE_DIR)

# custom code
import utils.utils
CONFIG = utils.utils.load_config("../../config.json")
import utils.papers

Using TensorFlow backend.


In [4]:
RANDOM_SEED = CONFIG['random_seed']
GROUPS = CONFIG['experiment_configs']['adult_bb']['groups']
EVAL_GROUPS = CONFIG['experiment_configs']['adult_bb']['eval_groups']
BATCH_SIZE = 32

print(RANDOM_SEED)
print(f"EVAL_GROUPS: {EVAL_GROUPS}")

PROCESSED_DIR = os.path.join(BASE_DIR, f'processed/adult_bb/rs={RANDOM_SEED}')
MODELS_DIR = os.path.join(BASE_DIR, f'models/adult_bb/rs={RANDOM_SEED}')

PROCESSED_SAVEPATH = utils.utils.get_savepath(PROCESSED_DIR, "adult_bb", ".pkl", g=GROUPS, eg=EVAL_GROUPS)
BASE_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, "adult_bb", ".h5", mt="base") # mt = model_type

# models saved here
if not os.path.exists(BASE_MODEL_SAVEPATH):
    print(f"warning: model has not been done for rs={RANDOM_SEED}")

55
EVAL_GROUPS: ['gender_Male', 'gender_Female']


In [5]:
dat = None
# load processed data. we pass groups = [] because processing is done per group choice
# and we can use any
with open(PROCESSED_SAVEPATH, 'rb') as f:
    dat = pickle.load(f)

x_train = dat['x_train']
y_train = dat['y_train']

x_hyper_train = dat['x_hyper_train']
y_hyper_train = dat['y_hyper_train']

x_val = dat['x_val']
y_val = dat['y_val']

x_test = dat['x_test']
y_test = dat['y_test']

In [6]:
y_train = tf.keras.utils.to_categorical(y_train)
y_hyper_train = tf.keras.utils.to_categorical(y_hyper_train)
y_val = tf.keras.utils.to_categorical(y_val)
y_test = tf.keras.utils.to_categorical(y_test)

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=x_train.shape[1]),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax),
])
model.load_weights(BASE_MODEL_SAVEPATH)

# Baseline 1: Fine-Tune

In [8]:
# reload save weights, in case being run out-of-order
model.load_weights(BASE_MODEL_SAVEPATH)

In [9]:
# create a "train" and "test" on the valid set
x_val_train, x_val_test, y_val_train, y_val_test = model_selection.train_test_split(
    x_val,
    y_val,
    test_size=0.33,
    stratify=y_val,
    random_state=RANDOM_SEED,
)

In [10]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Need to save the best model by validation loss
FT_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, "adult", ".h5", mt="ft")

save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=FT_MODEL_SAVEPATH,
    monitor="val_loss",
    mode='min',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
)

callbacks = [save_best]

In [12]:
model.fit(
    x_val_train,
    y_val_train,
    batch_size = BATCH_SIZE,
    epochs = 100,
    validation_data = (x_val_test, y_val_test),
    callbacks=callbacks,
)

Epoch 1/100
27/35 [======================>.......] - ETA: 0s - loss: 0.3152 - accuracy: 0.8438
Epoch 00001: val_loss improved from inf to 0.30103, saving model to ../../../models/adult_bb/rs=55/adult_mt=ft.h5
35/35 [==============================] - 0s 7ms/step - loss: 0.3067 - accuracy: 0.8495 - val_loss: 0.3010 - val_accuracy: 0.8699
Epoch 2/100
28/35 [=======================>......] - ETA: 0s - loss: 0.3009 - accuracy: 0.8538
Epoch 00002: val_loss improved from 0.30103 to 0.30086, saving model to ../../../models/adult_bb/rs=55/adult_mt=ft.h5
35/35 [==============================] - 0s 4ms/step - loss: 0.3065 - accuracy: 0.8505 - val_loss: 0.3009 - val_accuracy: 0.8680
Epoch 3/100
27/35 [======================>.......] - ETA: 0s - loss: 0.2941 - accuracy: 0.8530
Epoch 00003: val_loss did not improve from 0.30086
35/35 [==============================] - 0s 3ms/step - loss: 0.3063 - accuracy: 0.8514 - val_loss: 0.3018 - val_accuracy: 0.8680
Epoch 4/100
30/35 [========================>.

29/35 [=======================>......] - ETA: 0s - loss: 0.2954 - accuracy: 0.8653
Epoch 00029: val_loss did not improve from 0.30035
35/35 [==============================] - 0s 3ms/step - loss: 0.3021 - accuracy: 0.8606 - val_loss: 0.3014 - val_accuracy: 0.8662
Epoch 30/100
30/35 [========================>.....] - ETA: 0s - loss: 0.3093 - accuracy: 0.8573
Epoch 00030: val_loss did not improve from 0.30035
35/35 [==============================] - 0s 3ms/step - loss: 0.3020 - accuracy: 0.8606 - val_loss: 0.3014 - val_accuracy: 0.8662
Epoch 31/100
30/35 [========================>.....] - ETA: 0s - loss: 0.2992 - accuracy: 0.8656
Epoch 00031: val_loss did not improve from 0.30035
35/35 [==============================] - 0s 3ms/step - loss: 0.3019 - accuracy: 0.8606 - val_loss: 0.3020 - val_accuracy: 0.8662
Epoch 32/100
29/35 [=======================>......] - ETA: 0s - loss: 0.2997 - accuracy: 0.8567
Epoch 00032: val_loss did not improve from 0.30035
35/35 [==============================]

Epoch 59/100
29/35 [=======================>......] - ETA: 0s - loss: 0.2957 - accuracy: 0.8675
Epoch 00059: val_loss did not improve from 0.30035
35/35 [==============================] - 0s 3ms/step - loss: 0.2992 - accuracy: 0.8661 - val_loss: 0.3028 - val_accuracy: 0.8662
Epoch 60/100
29/35 [=======================>......] - ETA: 0s - loss: 0.2990 - accuracy: 0.8696
Epoch 00060: val_loss did not improve from 0.30035
35/35 [==============================] - 0s 3ms/step - loss: 0.2991 - accuracy: 0.8661 - val_loss: 0.3028 - val_accuracy: 0.8662
Epoch 61/100
29/35 [=======================>......] - ETA: 0s - loss: 0.2983 - accuracy: 0.8675
Epoch 00061: val_loss did not improve from 0.30035
35/35 [==============================] - 0s 3ms/step - loss: 0.2991 - accuracy: 0.8670 - val_loss: 0.3028 - val_accuracy: 0.8662
Epoch 62/100
30/35 [========================>.....] - ETA: 0s - loss: 0.2916 - accuracy: 0.8677
Epoch 00062: val_loss did not improve from 0.30035
35/35 [==================

Epoch 89/100
29/35 [=======================>......] - ETA: 0s - loss: 0.3044 - accuracy: 0.8653
Epoch 00089: val_loss did not improve from 0.30035
35/35 [==============================] - 0s 3ms/step - loss: 0.2973 - accuracy: 0.8688 - val_loss: 0.3044 - val_accuracy: 0.8606
Epoch 90/100
29/35 [=======================>......] - ETA: 0s - loss: 0.2982 - accuracy: 0.8642
Epoch 00090: val_loss did not improve from 0.30035
35/35 [==============================] - 0s 3ms/step - loss: 0.2972 - accuracy: 0.8688 - val_loss: 0.3047 - val_accuracy: 0.8625
Epoch 91/100
29/35 [=======================>......] - ETA: 0s - loss: 0.2935 - accuracy: 0.8696
Epoch 00091: val_loss did not improve from 0.30035
35/35 [==============================] - 0s 3ms/step - loss: 0.2971 - accuracy: 0.8688 - val_loss: 0.3048 - val_accuracy: 0.8606
Epoch 92/100
25/35 [====================>.........] - ETA: 0s - loss: 0.2766 - accuracy: 0.8737
Epoch 00092: val_loss did not improve from 0.30035
35/35 [==================

# Baseline 2: Forward Correct

Paper: https://arxiv.org/pdf/1609.03683.pdf

In [14]:
# reload trained weights
model.load_weights(BASE_MODEL_SAVEPATH)

In [15]:
x_train_full = np.concatenate([x_train, x_hyper_train])
y_train_full = np.concatenate([y_train, y_hyper_train])

In [16]:
# T[i,j] means the probabilty that class i is flipped to class j.
T_hat = utils.papers.forward_correct_est_T(
    model,
    x_train_full,
    nc=2,
    batch_size=BATCH_SIZE,
)

In [17]:
np.round(T_hat, decimals=2)

array([[1.  , 0.  ],
       [0.04, 0.96]])

In [18]:
loss = utils.papers.make_forward_correct_loss(T_hat)

In [19]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
# Need to save the best model by validation loss
FC_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, "adult", ".h5", mt="fc")

save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=FC_MODEL_SAVEPATH,
    monitor="val_loss",
    mode='min',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
)

callbacks = [save_best]

In [22]:
model.fit(
    x_train,
    y_train,
    batch_size = BATCH_SIZE,
    epochs = 100,
    validation_data = (x_hyper_train, y_hyper_train),
    callbacks=callbacks,
)

# Baseline 3: Learn to Weigh Examples
Paper: https://arxiv.org/pdf/1803.09050.pdf

This is a type of meta-learning, which doesn't quite work with the keras API. We will need to manually implement the training loop.

In [23]:
model.load_weights(BASE_MODEL_SAVEPATH)

In [24]:
optimizer = tf.keras.optimizers.SGD()

In [25]:
# Reduction.NONE means the cross entropy is computed per entry in the batch
# but is not aggregated. Traditional cross entropy will average the results.
ce = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

In [26]:
epochs = 100

In [28]:
best_loss = float('inf')
LRW_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, "adult_bb", ".h5", mt="lrw")

total_batches = len(x_train) // BATCH_SIZE 
if total_batches * BATCH_SIZE < len(x_train):
    # this usually happens as // operator rounds down
    total_batches += 1
    
# custom train loop
for epoch in range(epochs):
    # implements a train loop
    print(f"Epoch {epoch}:\n----------")
    
    loss_sum = 0
    for i in tqdm(range(total_batches)):
        # grab the batch and labels
        batch = x_train[i * BATCH_SIZE : i * BATCH_SIZE + BATCH_SIZE]
        labels = y_train[i * BATCH_SIZE : i * BATCH_SIZE + BATCH_SIZE]
        
        # most of the details are abstracted away in this function
        loss = utils.papers.train_step(model, batch, labels, x_val, y_val, ce, optimizer)
        loss_sum += loss

        # print ongoing avg loss
        print(f"Loss: {loss_sum / i}", end='\r')
    
    # compute validation accuracy
    preds = utils.utils.compute_preds(
        model,
        x_val,
        batch_size=BATCH_SIZE,
    )
    val_acc = (np.argmax(preds, axis=1) == np.argwhere(y_val)[:,1]).mean()
    loss_avg = loss_sum / total_batches
    
    print(f"Val Acc: {val_acc}")
    print(f"Val Loss: {loss_avg}", end='\n\n')
        
    # implements save best logic
    if loss_avg < best_loss:
        best_loss = loss_avg
        print(f"Saving new best weights to {LRW_MODEL_SAVEPATH}")
        model.save_weights(
            filepath=LRW_MODEL_SAVEPATH,
            save_format="h5",
        )
        last_best_epoch = epoch

# Baseline 4: Kernel Mean Matching

Paper: https://papers.nips.cc/paper/2006/file/a2186aa7c086b46ad4e8bf81e2a3a19b-Paper.pdf

In [29]:
model.load_weights(BASE_MODEL_SAVEPATH)

In [30]:
# the KMM algorithm does not scale well as inputs grow
# if we were to use the full train and test set, I don't even know
# how long it would take. Instead, we bunch the train into groups (randomly)
# of 1000 and apply KMM to the group and the full test set
# we then stitch together the estimated betas for the full train set
group_size = 1000
# these are a random arangement of indices
rand_inds = np.random.RandomState(seed=RANDOM_SEED).permutation( np.arange(len(x_train)) )
# these are the betas but ordered with respect to x_train
betas_ordered = np.zeros(len(x_train))

start_i = 0
end_i = start_i + group_size
while start_i < len(x_train):
    print(f"({start_i}-{end_i})")
    
    # grab the current group
    inds = rand_inds[start_i : end_i]
    
    kmm = utils.papers.KMM()
    # fit the group with the entire test data
    betas = kmm.fit(x_train[inds], x_test)
    # fill in betas_ordered at the indices in the current group
    betas_ordered[inds] = betas.reshape(-1) # flatten
    
    start_i = end_i
    end_i = start_i + group_size

(0-1000)
     pcost       dcost       gap    pres   dres
 0: -2.6357e+06 -2.6679e+06  2e+05  7e-02  3e-16
 1: -2.6355e+06 -2.6536e+06  4e+04  1e-02  4e-16
 2: -2.6347e+06 -2.6405e+06  1e+04  3e-03  3e-16
 3: -2.6340e+06 -2.6354e+06  2e+03  6e-04  4e-16
 4: -2.6337e+06 -2.6343e+06  9e+02  2e-04  4e-16
 5: -2.6337e+06 -2.6339e+06  3e+02  5e-05  4e-16
 6: -2.6337e+06 -2.6337e+06  4e+01  9e-16  4e-16
 7: -2.6337e+06 -2.6337e+06  2e+00  2e-16  4e-16
Optimal solution found.
(1000-2000)
     pcost       dcost       gap    pres   dres
 0: -2.6357e+06 -2.6677e+06  2e+05  7e-02  3e-16
 1: -2.6355e+06 -2.6529e+06  3e+04  1e-02  4e-16
 2: -2.6349e+06 -2.6412e+06  1e+04  3e-03  4e-16
 3: -2.6343e+06 -2.6361e+06  3e+03  9e-04  4e-16
 4: -2.6339e+06 -2.6343e+06  8e+02  2e-04  4e-16
 5: -2.6338e+06 -2.6339e+06  1e+02  2e-05  4e-16
 6: -2.6338e+06 -2.6338e+06  3e+00  5e-07  4e-16
 7: -2.6338e+06 -2.6338e+06  5e-02  7e-09  3e-16
Optimal solution found.
(2000-3000)
     pcost       dcost       gap    pre

     pcost       dcost       gap    pres   dres
 0: -2.6357e+06 -2.6679e+06  2e+05  7e-02  3e-16
 1: -2.6355e+06 -2.6533e+06  4e+04  1e-02  4e-16
 2: -2.6346e+06 -2.6410e+06  1e+04  3e-03  3e-16
 3: -2.6338e+06 -2.6355e+06  3e+03  9e-04  3e-16
 4: -2.6334e+06 -2.6337e+06  7e+02  2e-04  4e-16
 5: -2.6333e+06 -2.6333e+06  5e+01  1e-05  4e-16
 6: -2.6333e+06 -2.6333e+06  6e+00  1e-06  4e-16
 7: -2.6333e+06 -2.6333e+06  1e-01  3e-08  4e-16
Optimal solution found.
(18000-19000)
     pcost       dcost       gap    pres   dres
 0: -2.6357e+06 -2.6676e+06  2e+05  6e-02  4e-16
 1: -2.6356e+06 -2.6518e+06  3e+04  7e-03  4e-16
 2: -2.6350e+06 -2.6393e+06  6e+03  2e-03  3e-16
 3: -2.6347e+06 -2.6359e+06  2e+03  4e-04  4e-16
 4: -2.6345e+06 -2.6347e+06  1e+02  1e-15  4e-16
 5: -2.6346e+06 -2.6346e+06  5e+01  7e-17  3e-16
 6: -2.6346e+06 -2.6346e+06  1e+01  9e-16  4e-16
 7: -2.6346e+06 -2.6346e+06  4e-01  8e-16  4e-16
Optimal solution found.
(19000-20000)
     pcost       dcost       gap    pres   d

In [31]:
# save betas for later analysis, if any
KMM_BETAS_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, "adult_kmm_betas", ".npy")
np.save(
    file = KMM_BETAS_SAVEPATH,
    arr = betas_ordered,
)

In [32]:
# make a fresh model
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=x_train.shape[1]),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax),
])

In [33]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
# Need to save the best model by validation loss
KMM_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, "adult_bb", ".h5", mt="kmm")
save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=KMM_MODEL_SAVEPATH,
    monitor="val_loss",
    mode='min',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
)

callbacks = [save_best]

In [37]:
model.fit(
    x_train,
    y_train,
    batch_size = BATCH_SIZE,
    epochs = 100,
    validation_data = (x_val, y_val),
    callbacks=callbacks,
)

# Baseline 5: Just Train on Validation Set

In [38]:
# make a fresh model
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=x_train.shape[1]),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax),
])

In [39]:
# create a "train" and "test" on the valid set
x_val_train, x_val_test, y_val_train, y_val_test = model_selection.train_test_split(
    x_val,
    y_val,
    test_size=0.33,
    stratify=y_val,
    random_state=RANDOM_SEED,
)

In [40]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [42]:
# Need to save the best model by validation loss
JV_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, "adult_bb", ".h5", mt="jv")
save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=JV_MODEL_SAVEPATH,
    monitor="val_loss",
    mode='min',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
)

callbacks = [save_best]

In [43]:
model.fit(
    x_train,
    y_train,
    batch_size = BATCH_SIZE,
    epochs = 100,
    validation_data = (x_hyper_train, y_hyper_train),
    callbacks=callbacks,
)

Epoch 1/100
669/682 [============================>.] - ETA: 0s - loss: 0.4570 - accuracy: 0.7802
Epoch 00001: val_loss improved from inf to 0.40212, saving model to ../../../models/adult_bb/rs=55/adult_bb_mt=jv.h5
682/682 [==============================] - 2s 3ms/step - loss: 0.4556 - accuracy: 0.7808 - val_loss: 0.4021 - val_accuracy: 0.8146
Epoch 2/100
670/682 [============================>.] - ETA: 0s - loss: 0.3735 - accuracy: 0.8309
Epoch 00002: val_loss improved from 0.40212 to 0.37190, saving model to ../../../models/adult_bb/rs=55/adult_bb_mt=jv.h5
682/682 [==============================] - 2s 3ms/step - loss: 0.3731 - accuracy: 0.8312 - val_loss: 0.3719 - val_accuracy: 0.8340
Epoch 3/100
667/682 [============================>.] - ETA: 0s - loss: 0.3520 - accuracy: 0.8408
Epoch 00003: val_loss improved from 0.37190 to 0.35947, saving model to ../../../models/adult_bb/rs=55/adult_bb_mt=jv.h5
682/682 [==============================] - 2s 3ms/step - loss: 0.3524 - accuracy: 0.8402

Epoch 25/100
662/682 [============================>.] - ETA: 0s - loss: 0.3115 - accuracy: 0.8567
Epoch 00025: val_loss improved from 0.32836 to 0.32835, saving model to ../../../models/adult_bb/rs=55/adult_bb_mt=jv.h5
682/682 [==============================] - 2s 3ms/step - loss: 0.3112 - accuracy: 0.8568 - val_loss: 0.3284 - val_accuracy: 0.8481
Epoch 26/100
664/682 [============================>.] - ETA: 0s - loss: 0.3113 - accuracy: 0.8561
Epoch 00026: val_loss improved from 0.32835 to 0.32763, saving model to ../../../models/adult_bb/rs=55/adult_bb_mt=jv.h5
682/682 [==============================] - 2s 3ms/step - loss: 0.3109 - accuracy: 0.8563 - val_loss: 0.3276 - val_accuracy: 0.8494
Epoch 27/100
654/682 [===========================>..] - ETA: 0s - loss: 0.3115 - accuracy: 0.8558
Epoch 00027: val_loss improved from 0.32763 to 0.32744, saving model to ../../../models/adult_bb/rs=55/adult_bb_mt=jv.h5
682/682 [==============================] - 2s 3ms/step - loss: 0.3106 - accuracy:

Epoch 49/100
675/682 [============================>.] - ETA: 0s - loss: 0.3070 - accuracy: 0.8589
Epoch 00049: val_loss did not improve from 0.32500
682/682 [==============================] - 2s 3ms/step - loss: 0.3072 - accuracy: 0.8590 - val_loss: 0.3251 - val_accuracy: 0.8506
Epoch 50/100
657/682 [===========================>..] - ETA: 0s - loss: 0.3060 - accuracy: 0.8590
Epoch 00050: val_loss improved from 0.32500 to 0.32496, saving model to ../../../models/adult_bb/rs=55/adult_bb_mt=jv.h5
682/682 [==============================] - 2s 3ms/step - loss: 0.3071 - accuracy: 0.8581 - val_loss: 0.3250 - val_accuracy: 0.8508
Epoch 51/100
655/682 [===========================>..] - ETA: 0s - loss: 0.3058 - accuracy: 0.8595
Epoch 00051: val_loss improved from 0.32496 to 0.32481, saving model to ../../../models/adult_bb/rs=55/adult_bb_mt=jv.h5
682/682 [==============================] - 2s 3ms/step - loss: 0.3070 - accuracy: 0.8587 - val_loss: 0.3248 - val_accuracy: 0.8506
Epoch 52/100
665/682

Epoch 75/100
659/682 [===========================>..] - ETA: 0s - loss: 0.3066 - accuracy: 0.8601
Epoch 00075: val_loss did not improve from 0.32412
682/682 [==============================] - 2s 3ms/step - loss: 0.3057 - accuracy: 0.8604 - val_loss: 0.3242 - val_accuracy: 0.8505
Epoch 76/100
655/682 [===========================>..] - ETA: 0s - loss: 0.3049 - accuracy: 0.8607
Epoch 00076: val_loss did not improve from 0.32412
682/682 [==============================] - 2s 3ms/step - loss: 0.3057 - accuracy: 0.8600 - val_loss: 0.3241 - val_accuracy: 0.8510
Epoch 77/100
671/682 [============================>.] - ETA: 0s - loss: 0.3059 - accuracy: 0.8597
Epoch 00077: val_loss did not improve from 0.32412
682/682 [==============================] - 2s 3ms/step - loss: 0.3057 - accuracy: 0.8601 - val_loss: 0.3241 - val_accuracy: 0.8511
Epoch 78/100
674/682 [============================>.] - ETA: 0s - loss: 0.3061 - accuracy: 0.8598
Epoch 00078: val_loss improved from 0.32412 to 0.32408, saving